# ABOUT THIS NOTEBOOK
## Purpose
This notebook uses the best machine learning models that were obtained after refinement and builds and ensemble model.    
## Input
'data_set.pickle' generated by 'data_processing.ipynb'.
## Output
Results of residual analysis.
## Tasks Performed
* Load library packages
* Load pickle file
* Split data into train & test sets
    * Train: weeks 1 & 2, Test: week 3
    * Perform feature scaling
* Run the following algorithms:
    * Random Forest Regressor
    * Support Vector Machines
    * Gradient Boosting Regressor
    * Neural Networks
* Average the results to generate an ensemble

# LOAD LIBRARY PACKAGES

In [1]:
# Import the required library packages
import os
import re
import timeit

import numpy as np  
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow.contrib.learn as skflow

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

from six.moves import cPickle as pickle

# Settings for matplotlib, Seaborn
%matplotlib inline
sns.set_style('whitegrid')

# Set font sizes for matplots
plt.rcParams.update({'font.size': 15, 
                     'legend.fontsize': 'medium', 
                     'axes.titlesize': 'medium', 
                     'axes.labelsize': 'medium'})

print 'Read in packages from os, numpy, pandas, matplotlib, seaborn, tensorflow, sklearn & six'

Read in packages from os, numpy, pandas, matplotlib, seaborn, tensorflow, sklearn & six


# LOAD PICKLE FILE

In [2]:
pickle_file = 'data_set.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    pdata_set = save['data_set']
    del save
    print 'Loaded ptrain_set', pdata_set.shape
    
f.close()

Loaded ptrain_set (199584, 55)


In [3]:
pdata_set.columns.values

array(['district_id', 'num_day', 'time_slot', 'week_day', 'demand',
       'demand_t-1', 'demand_t-2', 'demand_t-3', 'supply', 'supply_t-1',
       'supply_t-2', 'supply_t-3', 'gap', 'weather', 'temperature',
       'pollution', 'poi_pc1', 'poi_pc2', 'poi_pc3', 'poi_pc4',
       'poi_cluster', 'tj_lvl1', 'tj_lvl2', 'tj_lvl3', 'tj_lvl4', 'dist_0',
       'dist_1', 'dist_2', 'dist_3', 'dist_4', 'dist_5', 'dist_6',
       'numday_0', 'numday_1', 'numday_2', 'numday_3', 'numday_4', 'ts_0',
       'ts_1', 'ts_2', 'ts_3', 'ts_4', 'ts_5', 'ts_6', 'ts_7', 'weekday_0',
       'weekday_1', 'weekday_2', 'poi_0', 'poi_1', 'poi_2', 'wthr_0',
       'wthr_1', 'wthr_2', 'wthr_3'], dtype=object)

# ADD GAP FOR PREVIOUS TIME SLOTS

In [ ]:
# Create new gap features for previous 3 time slots
pdata_set['gap_t-1'] = (pdata_set['demand_t-1'] - pdata_set['supply_t-1'])
pdata_set['gap_t-2'] = (pdata_set['demand_t-2'] - pdata_set['supply_t-2'])
pdata_set['gap_t-3'] = (pdata_set['demand_t-3'] - pdata_set['supply_t-3'])

# SPLIT DATA INTO TRAIN & TEST SETS

## Use weeks 1 & 2 for training, week 3 for test

In [4]:
train_days     = range(1,15)
test_days      = range(15, 22)

X_train     = pdata_set[(pdata_set['num_day'].isin(train_days))]
X_test      = pdata_set[(pdata_set['num_day'].isin(test_days))]

print "Shape of X_train, X_test:", X_train.shape, X_test.shape, "\n\n"

Shape of X_train, X_test: (133056, 55) (66528, 55) 




## Generate scaled features for train & test sets

In [5]:
import warnings
warnings.filterwarnings('ignore')

gap_predictors = ['demand_t-1', 'demand_t-2', 'demand_t-3',
                  'supply_t-1', 'supply_t-2', 'supply_t-3',
                  'poi_pc1', 'poi_pc2',
                  'tj_lvl1', 'tj_lvl2', 'tj_lvl3',
                  'ts_0', 'ts_1', 'ts_2', 'ts_3', 'ts_4', 'ts_5', 'ts_6', 'ts_7',
                  'pollution', 'temperature',
                  'wthr_0', 'wthr_1', 'wthr_2', 'wthr_3',
                  'gap_t-1', 'gap_t-2', 'gap_t-3',
                  'time_slot', 'week_day'
                 ] 
gX_train = []
gy_train = []
gX_test  = []
gy_test  = []

# Use StandardScaler to achieve zero mean and unit variance
# Generate two scalers: input and target
input_scaler = StandardScaler().fit(pdata_set[gap_predictors])
target_scaler = StandardScaler().fit(pdata_set['gap'])

# Scale both training & test data
gX_train  = input_scaler.transform(X_train[gap_predictors])
gy_train  = target_scaler.transform(X_train['gap'])

gX_test = input_scaler.transform(X_test[gap_predictors])
gy_test = target_scaler.transform(X_test['gap'])

## Define un-scaled features for train & test sets

In [ ]:
gXu_train = []
gyu_train = []
gXu_test  = []
gyu_test = []


# Define training & test data frames
gXu_train  = X_train[gap_predictors].as_matrix()
gyu_train  = X_train['gap'].as_matrix()

gXu_test = X_test[gap_predictors].as_matrix()
gyu_test = X_test['gap'].as_matrix()

# SCORING FUNCTION

In [ ]:
def print_score(y_train, y_pred_train, y_test, y_pred_test):
    
    """
    Present the MSE, R^2 and MAPE scores for train & test sets as a table.

    Parameters
    ----------
    y_train      : Array containing expected values for train set
    y_pred_train : Array containing predicted values for train set
    y_test       : Array containing expected values for test set
    y_pred_test  : Array containing predicted values for test set
    """
    
    m2score_train    = metrics.mean_squared_error(y_train,    y_pred_train)
    m2score_test     = metrics.mean_squared_error(y_test,     y_pred_test)


    r2score_train    = metrics.r2_score(y_train,    y_pred_train)
    r2score_test     = metrics.r2_score(y_test,     y_pred_test)

    # Assumes data is for 144 time slots, 14 days (train), 7 days (test)
    mpscore_train    = mape_score(y_train,    y_pred_train, ((144*14)-1))
    mpscore_test     = mape_score(y_test,     y_pred_test, ((144*7)-1))


    sets_list = ["TRAIN", "TEST"]

    m2_scores = [m2score_train, m2score_test]
    r2_scores = [r2score_train, r2score_test]
    mp_scores = [mpscore_train, mpscore_test]


    print '\t\tMEAN^2\t\tR2\t\tMAPE'

    for s, m, r, mp in zip(sets_list, m2_scores, r2_scores, mp_scores):
        print '{0:10}\t{1:.3f}\t\t{2:.3f}\t\t{3:.3f}' .format(s, m, r, mp)

# RANDOM FORESTS

In [ ]:
rf_predictors = [0,3,1,5,2,8,4,19,25,26,27,28,29]

regressor = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=12, max_features=1.0, max_leaf_nodes=None, 
                                  min_samples_leaf=5, min_samples_split=15, min_weight_fraction_leaf=0.0, n_estimators=300, 
                                  n_jobs=1, oob_score=False, random_state=0, verbose=0, warm_start=False)

# Fit
regressor.fit(gX_train[:, rf_predictors], gy_train)

# Predict
rf_trainpred = target_scaler.inverse_transform(regressor.predict(gX_train[:, rf_predictors]))
rf_testpred  = target_scaler.inverse_transform(regressor.predict(gX_test[:, rf_predictors]))

# SUPPORT VECTOR MACHINES

In [ ]:
svm_predictors = [0,3,1,4,25,26,28,29]

regressor = SVR(kernel='linear', C=1.0, epsilon=0.1, cache_size=10000)

# Fit
regressor.fit(gX_train[:, svm_predictors], gy_train)

# Predict
svm_trainpred = target_scaler.inverse_transform(regressor.predict(gX_train[:, svm_predictors]))
svm_testpred  = target_scaler.inverse_transform(regressor.predict(gX_test[:, svm_predictors]))

# GRADIENT BOOSTED TREES

In [ ]:
gbr_predictors = [0,3,2,1,20,9,5,19,25,26,27,28,29] 

regressor = GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.15, loss='ls',max_depth=5, max_features=None, 
                                max_leaf_nodes=None, min_samples_leaf=3, min_samples_split=8, min_weight_fraction_leaf=0.0, 
                                n_estimators=100, presort='auto', random_state=None, subsample=0.8, verbose=0,
                                warm_start=False)

# Fit
regressor.fit(gXu_train[:, gbr_predictors], gyu_train)

# Predict
gbr_trainpred = regressor.predict(gXu_train[:, gbr_predictors])
gbr_testpred  = regressor.predict(gXu_test[:, gbr_predictors])

# NEURAL NETS

In [ ]:
nn_predictors = [0,3,1,5,2,8,4,19,25,26,27,28,29]

# Set up optimizer, regressor
learning_rate=0.01 
hidden_units=[11] 
dropout=0.3 
steps=50000 
batch_size=3000

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
regressor = skflow.DNNRegressor(hidden_units=hidden_units, optimizer=optimizer, dropout=float(dropout))

# Fit
regressor.fit(gX_train[:, nn_predictors], gy_train, steps=steps, batch_size=batch_size)

# Predict
nn_trainpred = target_scaler.inverse_transform(regressor.predict(gX_train[:, nn_predictors]))
nn_testpred  = target_scaler.inverse_transform(regressor.predict(gX_test[:, nn_predictors]))

# ENSEMBLE

In [ ]:
train_prediction = (rf_trainpred + svm_trainpred + gbr_trainpred + nn_trainpred) / 4.0
test_prediction  = (rf_testpred + svm_testpred + gbr_testpred + nn_testpred) / 4.0
print_score(gy_train, train_prediction, gy_test, test_prediction) 